In [5]:
from datasets import load_dataset, load_from_disk
import wordninja
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="2"
import re
import glob
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

import json
import datasets
import transformers
from transformers import AutoTokenizer, default_data_collator, get_scheduler
from hf_transformers.src.transformers.models.bert.configuration_bert import BertConfig
from hf_transformers.src.transformers.models.bert.modeling_bert import BertForPreTraining
from torch.utils.data import DataLoader
from hf_transformers.src.transformers.models.bert.modeling_bert import BertForSequenceClassification
from torch.optim import AdamW

import torch
from torch.optim import AdamW
from torch.utils.data import DataLoader

import sys
import math

import collections


from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)


In [6]:
#WARNING
#cacheディレクトリの指定は絶対マウント先のフォルダにするように
#そうしないとdockerイメージを管理している研究室サーバーのルートディレクトリ(/)がパンパンになってしまう
#/が容量オーバーすると何も動かなくなって他の人に迷惑
!export HF_DATASETS_CACHE="/home/"

In [7]:
df_hdfs_without_duplicate = pd.read_csv("./logdata/hdfs/df_hdfs_without_duplicate.csv", usecols=["BlockId", "Label", "log"])

In [8]:
df_hdfs_without_duplicate["log"]

0         081109 203518 143 INFO dfs.DataNode$DataXceive...
1         081109 203520 142 INFO dfs.DataNode$DataXceive...
2         081109 203521 145 INFO dfs.DataNode$DataXceive...
3         081109 203523 143 INFO dfs.DataNode$DataXceive...
4         081109 203529 148 INFO dfs.DataNode$DataXceive...
                                ...                        
575056    081111 110351 27174 INFO dfs.DataNode$DataXcei...
575057    081111 110359 26685 INFO dfs.DataNode$DataXcei...
575058    081111 110402 27311 INFO dfs.DataNode$DataXcei...
575059    081111 110412 27231 INFO dfs.DataNode$DataXcei...
575060    081111 110413 26687 INFO dfs.DataNode$DataXcei...
Name: log, Length: 575061, dtype: object

In [9]:
#PCAテストデータの読み込み
df_hdfs_anomaly_test = pd.read_csv("./logdata/loglizer_hdfs_test.csv", usecols=["BlockId"])
pca_test_block_ids = list(df_hdfs_anomaly_test["BlockId"])
df_cmp_loglizer_test = df_hdfs_without_duplicate[df_hdfs_without_duplicate["BlockId"].isin(pca_test_block_ids)]
df_cmp_loglizer_test.to_csv("./logdata/hdfs/loglizer_hdfs_test_data.csv")

In [10]:
#PCA訓練データの読み込み
df_hdfs_anomaly_train = pd.read_csv("./logdata/loglizer_hdfs_train.csv", usecols=["BlockId"])
pca_train_block_ids = list(df_hdfs_anomaly_train["BlockId"])
df_cmp_loglizer_train = df_hdfs_without_duplicate[df_hdfs_without_duplicate["BlockId"].isin(pca_train_block_ids)]
df_cmp_loglizer_train.to_csv("./logdata/hdfs/loglizer_hdfs_train_data.csv")

In [11]:
df_cmp_loglizer_train

,BlockId,Label,log
0,blk_-1608999687919862906,Normal,081109 203518 143 INFO dfs.DataNode$DataXceive...
1,blk_7503483334202473044,Normal,081109 203520 142 INFO dfs.DataNode$DataXceive...
2,blk_-3544583377289625738,Anomaly,081109 203521 145 INFO dfs.DataNode$DataXceive...
3,blk_-9073992586687739851,Normal,081109 203523 143 INFO dfs.DataNode$DataXceive...
4,blk_7854771516489510256,Normal,081109 203529 148 INFO dfs.DataNode$DataXceive...
...,...,...,...
4714,blk_8608974154783629388,Anomaly,081109 204328 30 INFO dfs.FSNamesystem: BLOCK*...
4728,blk_8225451996519902271,Anomaly,081109 204330 30 INFO dfs.FSNamesystem: BLOCK*...
4732,blk_-5350492094942566244,Anomaly,081109 204330 34 INFO dfs.FSNamesystem: BLOCK*...
4780,blk_4438299806816545743,Anomaly,081109 204335 31 INFO dfs.FSNamesystem: BLOCK*...


In [89]:
dataset_summary["train"].to_pandas()

,input,summary
0,INFO dfs.DataNode$DataXceiver : Receiving bloc...,Receiving block src; blockMap updated; Verific...
1,INFO dfs.DataNode$DataXceiver : Receiving bloc...,Receiving block src; blockMap updated; Verific...
2,INFO dfs.DataNode$DataXceiver : Receiving bloc...,Receiving block src; blockMap updated; Verific...
3,INFO dfs.DataNode$DataXceiver : Receiving bloc...,Receiving block src; blockMap updated; Verific...
4,INFO dfs.DataNode$PacketResponder : PacketResp...,Receiving block src; blockMap updated; Verific...
...,...,...
11975,INFO storage.BlockManager : Removing RDD 12744,Removing RDD;
11976,INFO storage.BlockManager : Removing RDD 12740,Removing RDD;
11977,INFO storage.BlockManager : Removing RDD 12736,Removing RDD;
11978,INFO storage.BlockManager : Removing RDD 12732,Removing RDD;


In [12]:
df_cmp_loglizer_test

,BlockId,Label,log
3929,blk_942782950815104487,Normal,081109 204208 26 INFO dfs.FSNamesystem: BLOCK*...
3930,blk_1819115441742884087,Normal,081109 204208 26 INFO dfs.FSNamesystem: BLOCK*...
3931,blk_7082151245718002140,Normal,081109 204208 31 INFO dfs.FSNamesystem: BLOCK*...
3932,blk_-3727175694322392368,Normal,081109 204208 32 INFO dfs.FSNamesystem: BLOCK*...
3933,blk_-7300835887062415762,Normal,081109 204208 33 INFO dfs.FSNamesystem: BLOCK*...
...,...,...,...
7935,blk_-1445970677921829671,Normal,081109 204836 29 INFO dfs.FSNamesystem: BLOCK*...
7936,blk_-5943236831140622436,Normal,081109 204836 30 INFO dfs.FSNamesystem: BLOCK*...
7937,blk_-5039164935117450945,Normal,081109 204836 30 INFO dfs.FSNamesystem: BLOCK*...
7938,blk_7379833155074044619,Normal,081109 204836 33 INFO dfs.FSNamesystem: BLOCK*...


In [13]:
# def log2list(path):
#     with open(path, "r") as f:
#         lines = [line.rstrip("\n") for line in f]
#     return lines

# hdfs = log2list("./logdata/hdfs/HDFS.log")
# log_label_pairs = {}

# df_hdfs_logs = pd.DataFrame({"log" : hdfs})
# df_hdfs_logs["BlockId"] = df_hdfs_logs["log"].map(lambda line : re.search(r"blk_[-]*[0-9]+", line).group(0))
# df_hdfs = df_hdfs_anomaly.merge(df_hdfs_logs, how="left", on=["BlockId"])

# df_hdfs_without_duplicate = df_hdfs.drop_duplicates(["BlockId"])
# df_hdfs_without_duplicate.to_csv("./logdata/hdfs/df_hdfs_without_duplicate.csv")

In [14]:
# df_hdfs_without_duplicate["text"] = df_hdfs_without_duplicate["log"].map(preprocess_log)

In [15]:
import transformers
transformers.__version__

'4.31.0.dev0'

In [16]:
#分類と事前学習で異なるトークナイザを利用している？
#根拠: 例えばHDFSの異常検知の場合, 異常ラベルはログデータのブロック番号と紐づけられている

In [17]:
#NOTE
#ここで記号を区切ってwordninjaで塊除去を行っている
#なお, かなり時間がかかるので実行には注意が必要
#もう少し賢く書きたかったが, その書き方を思いつく時間とナイーブに実行する時間はどうせ同じぐらいだろう
def preprocess_log(text, remove_digits=True):
    text = text.replace('[', " ")
    text = text.replace(".", " ")
    text = text.replace(",", " ")
    text = text.replace(":", " ")
    text = text.replace("/", " ")
    text = text.replace(";", " ")
    text = text.replace("=", " ")
    text = text.replace("*", " ")
    text = text.replace("_", " ")
    text = text.replace("-", " ")
    text = text.lower()
    text = " ".join(wordninja.split(text))
    remove_num = lambda eg : " ".join([word for word in eg.split(" ") if not word.isdigit()])
    if remove_digits:
        return remove_num(text)
    return text;

def preprocess_log_batch(example, remove_digits=True):
    return {
        "text": [preprocess_log(l, remove_digits=remove_digits) for l in example["log"]]
    }

In [71]:
df_ad_hdfs = pd.read_csv("./logdata/hdfs/hdfs_log_label_pairs.csv", usecols=["log", "BlockId", "Label"])
# df_hdfs_without_duplicate["text"] = df_hdfs_without_duplicate["log"].map(preprocess_log)

In [74]:
df_ad_hdfs["Label"].value_counts(normalize=True)

Normal     0.974207
Anomaly    0.025793
Name: Label, dtype: float64

In [82]:
df_downsampled = df_ad_hdfs[df_ad_hdfs["Label"]=="Anomaly"]
df_normal_sampled = df_ad_hdfs[df_ad_hdfs["Label"]=="Normal"].sample(len(df_downsampled))
df_downsampled = pd.concat([df_downsampled, df_normal_sampled])
df_downsampled.to_csv("./logdata/downsampled_anomaly.csv")
df_downsampled

,log,BlockId,Label
291,081109 203521 145 INFO dfs.DataNode$DataXceive...,blk_-3544583377289625738,Anomaly
292,081109 203521 29 INFO dfs.FSNamesystem: BLOCK*...,blk_-3544583377289625738,Anomaly
293,081109 203522 144 INFO dfs.DataNode$DataXceive...,blk_-3544583377289625738,Anomaly
294,081109 203522 145 INFO dfs.DataNode$DataXceive...,blk_-3544583377289625738,Anomaly
295,081109 203522 147 INFO dfs.DataNode$PacketResp...,blk_-3544583377289625738,Anomaly
...,...,...,...
6605276,081111 030649 32 INFO dfs.FSNamesystem: BLOCK*...,blk_4241722502979736601,Normal
1485026,081110 010143 30 INFO dfs.FSNamesystem: BLOCK*...,blk_2863437573282244260,Normal
7182233,081111 040435 33 INFO dfs.FSNamesystem: BLOCK*...,blk_-8441598874045640693,Normal
5684416,081111 044332 29 INFO dfs.FSNamesystem: BLOCK*...,blk_795775988001366663,Normal


In [44]:
#NOTE
#Loading train test dataset used in loglizer here for the sake of fair comparison

TEST_DATA_FILE_PATH = "./logdata/hdfs/loglizer_hdfs_test_data.csv"
TRAIN_DATA_FILE_PATH = "./logdata/hdfs/loglizer_hdfs_train_data.csv"


dataset_hdfs_test = load_dataset("csv", data_files=TEST_DATA_FILE_PATH)
dataset_hdfs_test = dataset_hdfs_test.remove_columns(['Unnamed: 0'])

dataset_hdfs_train = load_dataset("csv", data_files=TRAIN_DATA_FILE_PATH)
dataset_hdfs_train = dataset_hdfs_train.remove_columns(['Unnamed: 0'])

train_valid = dataset_hdfs_train["train"].train_test_split(0.2)
train_data = train_valid["train"]
eval_dataset = train_valid["test"]

Found cached dataset csv (/root/.cache/huggingface/datasets/csv/default-a438c5b2cd2acdf4/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d)
100%|██████████| 1/1 [00:00<00:00, 701.51it/s]
Found cached dataset csv (/root/.cache/huggingface/datasets/csv/default-da9900ce5058004e/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d)
100%|██████████| 1/1 [00:00<00:00, 487.77it/s]


In [20]:
#NOTE
#Removing all test data here

df_hdfs_without_duplicate = pd.read_csv("./logdata/hdfs/df_hdfs_without_duplicate.csv", usecols=["log", "BlockId", "Label"])
df_test = pd.read_csv(TEST_DATA_FILE_PATH, usecols=["log", "BlockId", "Label"])
test_blk_ids = list(df_test["BlockId"])
df_clean = df_hdfs_without_duplicate[~df_hdfs_without_duplicate["BlockId"].isin(test_blk_ids)]
df_clean.to_csv("./logdata/hdfs/anomaly_without_test.csv")

In [23]:
#Loading larger dataset that does not contain test data used in loglizer 
# datased_ad_hdfs = load_dataset("csv", data_files="./logdata/hdfs/anomaly_without_test.csv")
# datased_ad_hdfs = datased_ad_hdfs["train"].train_test_split(0.2)
# datased_ad_hdfs = datased_ad_hdfs.remove_columns(['Unnamed: 0'])

# train_data = datased_ad_hdfs["train"]
# eval_dataset = datased_ad_hdfs["test"]

In [45]:
train_data = train_data.map(
    preprocess_log_batch,
    batched=True,
)

In [46]:
eval_dataset = eval_dataset.map(
    preprocess_log_batch,
    batched=True,
)

In [47]:
log_tokenizer = AutoTokenizer.from_pretrained("./tokenizers/log_tokenizer_from_old_large/")
log_tokenizer_w_n = AutoTokenizer.from_pretrained("./tokenizers/log_tokenizer_from_old_without_numbers/")

batch_size=16  # change to 16 for full training

def process_data_to_model_inputs(batch):
    # tokenize the inputs and labels
    batch["labels"] = list(map(lambda x : int(x != "Normal"), batch["Label"]))

    return log_tokenizer_w_n(batch["text"], padding="max_length", truncation=True, max_length=180)



In [48]:
tokenized_train_data = train_data.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=1000, 
    remove_columns=['log', 'BlockId', 'Label', 'text']
)
tokenized_train_data

Dataset({
    features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 3175
})

In [49]:
tokenized_train_data

Dataset({
    features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 3175
})

In [50]:
tokenized_train_data.set_format(type="torch")

In [53]:
# small_train_dataset = tokenized_train_data.shuffle(seed=42).select(range(1000))


In [51]:
eval_dataset = eval_dataset.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=['log', 'BlockId', 'Label', 'text']
)

eval_dataset.set_format(type="torch")
# small_eval_dataset = eval_dataset.shuffle(seed=42).select(range(1000))

In [54]:
dataset_hdfs_test = load_dataset("csv", data_files=TEST_DATA_FILE_PATH)
dataset_hdfs_test = dataset_hdfs_test.remove_columns(['Unnamed: 0'])
test_dataset = dataset_hdfs_test.map(
    preprocess_log_batch,
    batched=True,
)["train"]

test_dataset = test_dataset.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=['log', 'BlockId', 'Label', 'text']
)


test_dataset.set_format(type="torch")
test_dataloader = DataLoader(test_dataset, batch_size=8)

Found cached dataset csv (/root/.cache/huggingface/datasets/csv/default-a438c5b2cd2acdf4/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d)
100%|██████████| 1/1 [00:00<00:00, 604.63it/s]
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-a438c5b2cd2acdf4/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d/cache-af2542df15b9a134.arrow


In [55]:

train_dataloader = DataLoader(tokenized_train_data, shuffle=True, batch_size=32)
eval_dataloader = DataLoader(eval_dataset, batch_size=8)

In [56]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [57]:


unilogConfigmodel = BertForSequenceClassification.from_pretrained("./logdata/unilog_pretrain_preln_on_attentions_0/", num_labels=2).to(device)
optimizer = AdamW(unilogConfigmodel.parameters(), lr=5e-5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ./logdata/unilog_pretrain_preln_on_attentions_0/ and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [58]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [59]:
num_training_steps

300

In [60]:
import evaluate

In [70]:
train_steps = 0

save_steps = 100

best_f1 = 0.0

for epoch in range(num_epochs):
    unilogConfigmodel.train()
    pbar = tqdm(train_dataloader)
    for batch in train_dataloader:
        train_steps += 1
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = unilogConfigmodel(**batch)
        loss = outputs.loss
        loss.backward()
        pbar.write(f"Loss:{loss}")
        pbar.update(1)
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        if train_steps % save_steps == 0:
            unilogConfigmodel.save_pretrained(f"./logdata/log_anomaly_pca_cmp_{train_steps}")
    import evaluate

    metric_acc = evaluate.load("accuracy")
    metric_f1 = evaluate.load("f1")
    metric_precision = evaluate.load("precision")
    metric_recall = evaluate.load("recall")

    unilogConfigmodel.eval()
    for batch in tqdm(eval_dataloader):
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = unilogConfigmodel(**batch)

        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        metric_acc.add_batch(predictions=predictions, references=batch["labels"])
        metric_f1.add_batch(predictions=predictions, references=batch["labels"])
        metric_precision.add_batch(predictions=predictions, references=batch["labels"])
        metric_recall.add_batch(predictions=predictions, references=batch["labels"])

    pbar.write(str(metric_acc.compute()))
    pbar.write(str(metric_precision.compute()))
    pbar.write(str(metric_recall.compute()))
    
    curr_f1_score = metric_f1.compute()["f1"]
    prev_f1_score = best_f1
    if (curr_f1_score > prev_f1_score):
        best_f1 = max(best_f1, curr_f1_score)
        unilogConfigmodel.save_pretrained(f"./logdata/log_anomaly_model_small_best_0708")


In [49]:
train_dataloader.dataset

Dataset({
    features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 3175
})

In [62]:
test_dataloader.dataset

Dataset({
    features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 3971
})

In [63]:
import evaluate

metric_acc = evaluate.load("accuracy")
metric_f1 = evaluate.load("f1")
metric_precision = evaluate.load("precision")
metric_recall = evaluate.load("recall")

unilogBestModel = BertForSequenceClassification.from_pretrained("./logdata/log_anomaly_pca_cmp_300/", num_labels=2).to(device)
unilogBestModel.eval()
for batch in tqdm(test_dataloader):
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = unilogBestModel(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric_acc.add_batch(predictions=predictions, references=batch["labels"])
    metric_f1.add_batch(predictions=predictions, references=batch["labels"])
    metric_precision.add_batch(predictions=predictions, references=batch["labels"])
    metric_recall.add_batch(predictions=predictions, references=batch["labels"])
    
print(metric_acc.compute())
print(metric_f1.compute())
print(metric_precision.compute())
print(metric_recall.compute())


100%|██████████| 497/497 [00:03<00:00, 142.78it/s]

{'accuracy': 0.9604633593553261}
{'f1': 0.0}
{'precision': 0.0}
{'recall': 0.0}



/usr/local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [69]:
test_dataloader.dataset.to_pandas()["labels"].value_counts(normalize=True)

0    0.960463
1    0.039537
Name: labels, dtype: float64

In [37]:
test_dataloader.dataset.to_pandas()["labels"].value_counts(normalize=True)

1    0.960463
0    0.039537
Name: labels, dtype: float64

In [19]:
print("""{'accuracy': 0.9604633593553261}
{'f1': 0.9798330122029544}
{'precision': 0.9604633593553261}
{'recall': 1.0}""")

{'accuracy': 0.9604633593553261}
{'f1': 0.9798330122029544}
{'precision': 0.9604633593553261}
{'recall': 1.0}
